# 크롤링 함수

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
import os
import csv
import io

# ==================================
# 1. 환경 설정 및 상수 정의
# ==================================

# 🚩 필터 기준: 게시일 >= 2024-11-01
START_DATE_FILTER = datetime(2024, 11, 1)
START_DATE_REQUEST = START_DATE_FILTER.strftime("%Y-%m-%d")

# 현재 날짜 (조회 종료일)
TODAY_DATE_REQUEST = datetime.now().strftime("%Y-%m-%d")

BASE_URL = "https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancList.do"
DETAIL_URL_BASE = "https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?mi=1026"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
    "Referer": "https://apply.lh.or.kr/"
}

# Payload 템플릿
FORM_DATA_TEMPLATE = {
    'currPage': 1,
    'panId': '',
    'cnpCd': '',
    'prevListCo': 50,
    'srchAisTpCd': '',
    'panSs': '',
    'startDt': START_DATE_REQUEST,
    'endDt': TODAY_DATE_REQUEST,
    'aisTpCd': '',
    'uppAisTpCd': '',
    'mi': '',
    'srchUppAisTpCd': '',
    'srchY': 'N',
    'srchFilter': 'N',
    'csCd': '',
    'CNP_CD': '',
    'ccrCnntSysDsCd': '',
    'xssChk': 'N',
    'panEdDt': TODAY_DATE_REQUEST.replace('-', ''),
    'listCo': 50,
    'panNm': '',
    'indVal': 'N',
    'maxSn': 50,
    'mvinQf': '',
    'netbgn': '',
    'viewType': '',
    'panStDt': '',
    'minSn': 0,
    'schTy': '0',
    'page': 1,
}

# 🚩 임대/분양 유형 목록
LEASE_TYPES = [
    "통합공공임대", "통합공공임대(신혼희망)", "국민임대", "공공임대", "영구임대", "행복주택",
    "행복주택(신혼희망)", "장기전세", "신축다세대매입임대", "가정어린이집", "매입임대",
    "전세임대", "집주인임대", "6년 공공임대주택"
]

SALE_TYPES = [
    "분양주택", "공공분양(신혼희망)"
]

def generate_detail_url(anchor_tag):
    """HTML 요소에서 상세 페이지 조회에 필요한 data 속성을 추출하여 URL을 생성합니다."""
    data_panId = anchor_tag.get('data-id1', '')
    data_ccr = anchor_tag.get('data-id2', '')
    data_upp = anchor_tag.get('data-id3', '')
    data_ais = anchor_tag.get('data-id4', '')

    if not all([data_panId, data_ccr, data_upp, data_ais]):
        return ""

    url = (
        f"{DETAIL_URL_BASE}&"
        f"panId={data_panId}&"
        f"ccrCnntSysDsCd={data_ccr}&"
        f"uppAisTpCd={data_upp}&"
        f"aisTpCd={data_ais}"
    )
    return url

# ==================================
# 2. 크롤링 핵심 함수
# ==================================

def crawl_lh_notices_all_data():
    """LH 홈페이지에서 2024-11-01 이후의 모든 공고 데이터를 크롤링합니다."""
    all_data = []
    page = 1
    list_count = FORM_DATA_TEMPLATE['listCo']

    print("🚀 LH 공고 데이터 전체 크롤링 시작 (POST 방식)...")
    print(f"**필터 기준: [모든 유형] + 게시일 {FORM_DATA_TEMPLATE['startDt']} 이후 데이터 수집.**")

    while True:
        form_data = FORM_DATA_TEMPLATE.copy()
        form_data['currPage'] = str(page)
        form_data['page'] = str(page)
        form_data['minSn'] = str((page - 1) * list_count)
        form_data['maxSn'] = str(page * list_count)
        form_data['prevListCo'] = str(list_count)

        print(f"\n📄 Crawling page {page} ({form_data['minSn']} to {form_data['maxSn']})...")

        try:
            response = requests.post(BASE_URL, data=form_data, headers=HEADERS, timeout=15)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            rows = soup.select("div.bbs_ListA table tbody tr")
            raw_data_count = len(rows)

            if raw_data_count == 0:
                print("🏁 데이터 없음. 목록의 끝에 도달했습니다. 크롤링 종료.")
                break

            new_data_count = 0
            stop_crawling = False

            for row in rows:
                try:
                    cols = [c.get_text(strip=True) for c in row.select("td")]
                    if len(cols) < 9:
                        continue

                    status = cols[7]
                    post_date_str = cols[5]

                    try:
                        post_date = datetime.strptime(post_date_str, '%Y.%m.%d')
                        if post_date < START_DATE_FILTER:
                            print(f"🚩 게시일 {post_date_str} 데이터가 필터 기준({START_DATE_REQUEST})보다 이전입니다. 추가 크롤링을 중단합니다.")
                            stop_crawling = True
                            break
                    except ValueError:
                        pass

                    title_anchor = row.select_one("td:nth-child(3) a")
                    em_tag = title_anchor.find('em')
                    if em_tag:
                        em_tag.extract()
                    title = title_anchor.get_text(strip=True).replace('\n', ' ')

                    detail_url = generate_detail_url(title_anchor)
                    if not detail_url:
                        continue

                    all_data.append({
                        '번호': cols[0],
                        '유형': cols[1],
                        '공고명': title,
                        '지역': cols[3],
                        '게시일': post_date_str,
                        '마감일': cols[6],
                        '상태': status,
                        '조회수': cols[8].replace(',', ''),
                        'URL': detail_url
                    })
                    new_data_count += 1
                except Exception:
                    continue

            print(f"✅ 페이지 {page} 로드 성공. {len(rows)}개 중 {new_data_count}개 데이터 추출.")

            if stop_crawling or raw_data_count < list_count:
                print("🏁 크롤링 종료 조건 충족. 전체 크롤링 종료.")
                break

            page += 1
            time.sleep(1)

        except requests.exceptions.RequestException as e:
            print(f"🚨 네트워크 또는 HTTP 요청 오류 발생: {e}")
            break
        except Exception as e:
            print(f"🚨 치명적인 오류 발생: {e}")
            break

    if all_data:
        print(f"\n✨ 전체 크롤링 완료. 총 {len(all_data)}건의 데이터를 수집했습니다.")
        return pd.DataFrame(all_data)
    else:
        print("🎉 완료! 수집된 데이터가 없습니다.")
        return pd.DataFrame()

# ==================================
# 3. 데이터 필터링, ID 부여 및 저장 함수
# ==================================

def filter_assign_id_and_save(df, types_list, id_prefix, output_filename):
    """데이터프레임을 특정 유형으로 필터링하고, ID를 부여한 뒤 CSV 파일로 저장합니다."""
    print(f"\n--- 🛠️ {id_prefix.upper()} 데이터 처리 시작 ---")

    df_filtered = df[df['유형'].isin(types_list)].copy()
    print(f"✅ '{id_prefix.capitalize()}' 유형 필터링 완료: 총 {len(df_filtered)}건")

    if df_filtered.empty:
        print(f"🚨 필터링 결과, 해당하는 '{id_prefix.capitalize()}' 데이터가 없습니다. 파일 저장 Skip.")
        return

    df_filtered['번호'] = pd.to_numeric(df_filtered['번호'], errors='coerce').fillna(-1).astype(int)
    df_filtered['게시일_DT'] = pd.to_datetime(df_filtered['게시일'], format='%Y.%m.%d', errors='coerce')
    df_filtered.dropna(subset=['게시일_DT'], inplace=True)

    df_sorted = df_filtered.sort_values(by=['게시일_DT', '번호'], ascending=[True, True]).reset_index(drop=True)
    df_sorted['ID'] = f"LH_{id_prefix}_" + (df_sorted.index + 1).astype(str)

    final_columns = ['ID'] + [col for col in df_sorted.columns if col not in ['ID', '게시일_DT']]
    df_final = df_sorted[final_columns]

    try:
        df_final.to_csv(output_filename, index=False, encoding='utf-8-sig')
        print(f"🎉 작업 완료! {id_prefix.capitalize()} 데이터 {len(df_final)}건 저장됨.")
        print(f"파일명: {output_filename}")
    except Exception as e:
        print(f"❌ CSV 파일 저장 중 오류 발생: {e}")

# ==================================
# 4. 통합 실행 함수
# ==================================

def crawl_and_save_lh_notices():
    """LH 공고를 크롤링하고 임대/분양으로 나누어 ID를 부여하고 파일로 저장하는 통합 함수."""
    df_all_notices = crawl_lh_notices_all_data()

    if df_all_notices.empty:
        print("\n=== 🛑 통합 프로세스 종료: 수집된 데이터가 없어 추가 작업이 없습니다. ===")
        return

    filter_assign_id_and_save(
        df=df_all_notices,
        types_list=LEASE_TYPES,
        id_prefix="lease",
        output_filename="lh_lease_notices_F.csv"
    )

    filter_assign_id_and_save(
        df=df_all_notices,
        types_list=SALE_TYPES,
        id_prefix="sale",
        output_filename="lh_sale_notices_F.csv"
    )

    print("\n\n=== ✅ 통합 프로세스 성공적으로 완료! ===")

# ==================================
# 5. 실행
# ==================================

if __name__ == "__main__":
    crawl_and_save_lh_notices()

🚀 LH 공고 데이터 전체 크롤링 시작 (POST 방식)...
**필터 기준: [모든 유형] + 게시일 2024-11-01 이후 데이터 수집.**

📄 Crawling page 1 (0 to 50)...
✅ 페이지 1 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 2 (50 to 100)...
✅ 페이지 2 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 3 (100 to 150)...
✅ 페이지 3 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 4 (150 to 200)...
✅ 페이지 4 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 5 (200 to 250)...
✅ 페이지 5 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 6 (250 to 300)...
✅ 페이지 6 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 7 (300 to 350)...
✅ 페이지 7 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 8 (350 to 400)...
✅ 페이지 8 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 9 (400 to 450)...
✅ 페이지 9 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 10 (450 to 500)...
✅ 페이지 10 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 11 (500 to 550)...
✅ 페이지 11 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 12 (550 to 600)...
✅ 페이지 12 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 13 (600 to 650)...
✅ 페이지 13 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 14 (650 to 70

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
import os
import csv
import io

# ==================================
# 1. 환경 설정 및 상수 정의
# ==================================

# 🚩 필터 기준: 게시일 >= 2024-11-01
START_DATE_FILTER = datetime(2024, 11, 1)
START_DATE_REQUEST = START_DATE_FILTER.strftime("%Y-%m-%d")

# 현재 날짜 (조회 종료일)
TODAY_DATE_REQUEST = datetime.now().strftime("%Y-%m-%d")

BASE_URL = "https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancList.do"
DETAIL_URL_BASE = "https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?mi=1026"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
    "Referer": "https://apply.lh.or.kr/"
}

# Payload 템플릿
FORM_DATA_TEMPLATE = {
    'currPage': 1,
    'panId': '',
    'cnpCd': '',
    'prevListCo': 50,
    'srchAisTpCd': '',
    'panSs': '',
    'startDt': START_DATE_REQUEST,
    'endDt': TODAY_DATE_REQUEST,
    'aisTpCd': '',
    'uppAisTpCd': '',
    'mi': '',
    'srchUppAisTpCd': '',
    'srchY': 'N',
    'srchFilter': 'N',
    'csCd': '',
    'CNP_CD': '',
    'ccrCnntSysDsCd': '',
    'xssChk': 'N',
    'panEdDt': TODAY_DATE_REQUEST.replace('-', ''),
    'listCo': 50,
    'panNm': '',
    'indVal': 'N',
    'maxSn': 50,
    'mvinQf': '',
    'netbgn': '',
    'viewType': '',
    'panStDt': '',
    'minSn': 0,
    'schTy': '0',
    'page': 1,
}

# 🚩 임대/분양 유형 목록
LEASE_TYPES = [
    "통합공공임대", "통합공공임대(신혼희망)", "국민임대", "공공임대", "영구임대", "행복주택",
    "행복주택(신혼희망)", "장기전세", "신축다세대매입임대", "가정어린이집", "매입임대",
    "전세임대", "집주인임대", "6년 공공임대주택"
]

SALE_TYPES = [
    "분양주택", "공공분양(신혼희망)"
]

def generate_detail_url(anchor_tag):
    """HTML 요소에서 상세 페이지 조회에 필요한 data 속성을 추출하여 URL을 생성합니다."""
    data_panId = anchor_tag.get('data-id1', '')
    data_ccr = anchor_tag.get('data-id2', '')
    data_upp = anchor_tag.get('data-id3', '')
    data_ais = anchor_tag.get('data-id4', '')

    if not all([data_panId, data_ccr, data_upp, data_ais]):
        return ""

    url = (
        f"{DETAIL_URL_BASE}&"
        f"panId={data_panId}&"
        f"ccrCnntSysDsCd={data_ccr}&"
        f"uppAisTpCd={data_upp}&"
        f"aisTpCd={data_ais}"
    )
    return url

# ==================================
# 2. 크롤링 핵심 함수
# ==================================

def crawl_lh_notices_all_data():
    """LH 홈페이지에서 2024-11-01 이후의 모든 공고 데이터를 크롤링합니다."""
    all_data = []
    page = 1
    list_count = FORM_DATA_TEMPLATE['listCo']

    print("🚀 LH 공고 데이터 전체 크롤링 시작 (POST 방식)...")
    print(f"**필터 기준: [모든 유형] + 게시일 {FORM_DATA_TEMPLATE['startDt']} 이후 데이터 수집.**")

    while True:
        form_data = FORM_DATA_TEMPLATE.copy()
        form_data['currPage'] = str(page)
        form_data['page'] = str(page)
        form_data['minSn'] = str((page - 1) * list_count)
        form_data['maxSn'] = str(page * list_count)
        form_data['prevListCo'] = str(list_count)

        print(f"\n📄 Crawling page {page} ({form_data['minSn']} to {form_data['maxSn']})...")

        try:
            response = requests.post(BASE_URL, data=form_data, headers=HEADERS, timeout=15)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            rows = soup.select("div.bbs_ListA table tbody tr")
            raw_data_count = len(rows)

            if raw_data_count == 0:
                print("🏁 데이터 없음. 목록의 끝에 도달했습니다. 크롤링 종료.")
                break

            new_data_count = 0
            stop_crawling = False

            for row in rows:
                try:
                    cols = [c.get_text(strip=True) for c in row.select("td")]
                    if len(cols) < 9:
                        continue

                    status = cols[7]
                    post_date_str = cols[5]

                    try:
                        # 게시일 필터링 로직 (최신순 가정)
                        post_date = datetime.strptime(post_date_str, '%Y.%m.%d')
                        if post_date < START_DATE_FILTER:
                            print(f"🚩 게시일 {post_date_str} 데이터가 필터 기준({START_DATE_REQUEST})보다 이전입니다. 추가 크롤링을 중단합니다.")
                            stop_crawling = True
                            break
                    except ValueError:
                        pass # 날짜 형식이 아니면 일단 수집

                    title_anchor = row.select_one("td:nth-child(3) a")
                    em_tag = title_anchor.find('em')
                    if em_tag:
                        em_tag.extract()
                    title = title_anchor.get_text(strip=True).replace('\n', ' ')

                    detail_url = generate_detail_url(title_anchor)
                    if not detail_url:
                        continue

                    all_data.append({
                        '번호': cols[0], # 크롤링 시에는 '번호' 컬럼을 수집 (정렬에 사용)
                        '유형': cols[1],
                        '공고명': title,
                        '지역': cols[3],
                        '게시일': post_date_str,
                        '마감일': cols[6],
                        '상태': status,
                        '조회수': cols[8].replace(',', ''),
                        'URL': detail_url
                    })
                    new_data_count += 1
                except Exception:
                    continue

            print(f"✅ 페이지 {page} 로드 성공. {len(rows)}개 중 {new_data_count}개 데이터 추출.")

            if stop_crawling or raw_data_count < list_count:
                print("🏁 크롤링 종료 조건 충족. 전체 크롤링 종료.")
                break

            page += 1
            time.sleep(1)

        except requests.exceptions.RequestException as e:
            print(f"🚨 네트워크 또는 HTTP 요청 오류 발생: {e}")
            break
        except Exception as e:
            print(f"🚨 치명적인 오류 발생: {e}")
            break

    if all_data:
        print(f"\n✨ 전체 크롤링 완료. 총 {len(all_data)}건의 데이터를 수집했습니다.")
        return pd.DataFrame(all_data)
    else:
        print("🎉 완료! 수집된 데이터가 없습니다.")
        return pd.DataFrame()

# ==================================
# 3. 데이터 필터링, ID 부여 및 저장 함수
# ==================================

def filter_assign_id_and_save(df, types_list, id_prefix, output_filename):
    """
    데이터프레임을 특정 유형으로 필터링하고, ID를 부여한 뒤 CSV 파일로 저장합니다.
    '번호' 컬럼은 최종 저장 시 제외됩니다.
    """
    print(f"\n--- 🛠️ {id_prefix.upper()} 데이터 처리 시작 ---")

    df_filtered = df[df['유형'].isin(types_list)].copy()
    print(f"✅ '{id_prefix.capitalize()}' 유형 필터링 완료: 총 {len(df_filtered)}건")

    if df_filtered.empty:
        print(f"🚨 필터링 결과, 해당하는 '{id_prefix.capitalize()}' 데이터가 없습니다. 파일 저장 Skip.")
        return

    # '번호' 컬럼은 정렬을 위해 사용
    df_filtered['번호'] = pd.to_numeric(df_filtered['번호'], errors='coerce').fillna(-1).astype(int)
    df_filtered['게시일_DT'] = pd.to_datetime(df_filtered['게시일'], format='%Y.%m.%d', errors='coerce')
    df_filtered.dropna(subset=['게시일_DT'], inplace=True)

    # 정렬 기준: 게시일_DT(오름차순) -> 번호(오름차순)
    df_sorted = df_filtered.sort_values(
        by=['게시일_DT', '번호'],
        ascending=[True, True]
    ).reset_index(drop=True)

    # 'ID' 컬럼 생성
    df_sorted['ID'] = f"LH_{id_prefix}_" + (df_sorted.index + 1).astype(str)

    # 🚩 최종 정리: 'ID'를 맨 앞으로, '게시일_DT'와 **'번호'** 컬럼을 제외
    columns_to_exclude = ['ID', '게시일_DT', '번호']
    final_columns = ['ID'] + [col for col in df_sorted.columns if col not in columns_to_exclude]
    df_final = df_sorted[final_columns]

    # CSV 파일로 저장
    try:
        df_final.to_csv(output_filename, index=False, encoding='utf-8-sig')
        print(f"🎉 작업 완료! {id_prefix.capitalize()} 데이터 {len(df_final)}건 저장됨.")
        print(f"파일명: {output_filename}")
    except Exception as e:
        print(f"❌ CSV 파일 저장 중 오류 발생: {e}")

# ==================================
# 4. 통합 실행 함수
# ==================================

def crawl_and_save_lh_notices():
    """LH 공고를 크롤링하고 임대/분양으로 나누어 ID를 부여하고 파일로 저장하는 통합 함수."""
    df_all_notices = crawl_lh_notices_all_data()

    if df_all_notices.empty:
        print("\n=== 🛑 통합 프로세스 종료: 수집된 데이터가 없어 추가 작업이 없습니다. ===")
        return

    # 2. 임대 공고 필터링 및 저장
    filter_assign_id_and_save(
        df=df_all_notices,
        types_list=LEASE_TYPES,
        id_prefix="lease",
        output_filename="lh_lease_notices_F.csv"
    )

    # 3. 분양 공고 필터링 및 저장
    filter_assign_id_and_save(
        df=df_all_notices,
        types_list=SALE_TYPES,
        id_prefix="sale",
        output_filename="lh_sale_notices_F.csv"
    )

    print("\n\n=== ✅ 통합 프로세스 성공적으로 완료! ===")

# ==================================
# 5. 실행
# ==================================

if __name__ == "__main__":
    crawl_and_save_lh_notices()

🚀 LH 공고 데이터 전체 크롤링 시작 (POST 방식)...
**필터 기준: [모든 유형] + 게시일 2024-11-01 이후 데이터 수집.**

📄 Crawling page 1 (0 to 50)...
✅ 페이지 1 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 2 (50 to 100)...
✅ 페이지 2 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 3 (100 to 150)...
✅ 페이지 3 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 4 (150 to 200)...
✅ 페이지 4 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 5 (200 to 250)...
✅ 페이지 5 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 6 (250 to 300)...
✅ 페이지 6 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 7 (300 to 350)...
✅ 페이지 7 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 8 (350 to 400)...
✅ 페이지 8 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 9 (400 to 450)...
✅ 페이지 9 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 10 (450 to 500)...
✅ 페이지 10 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 11 (500 to 550)...
✅ 페이지 11 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 12 (550 to 600)...
✅ 페이지 12 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 13 (600 to 650)...
✅ 페이지 13 로드 성공. 50개 중 50개 데이터 추출.

📄 Crawling page 14 (650 to 70

# 다운로드 함수

- 다운로드 및 lh_lease|sale_notices-download.F.csv 만들기

In [6]:
import requests
import os
import re
import pandas as pd
import time
from datetime import datetime
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import warnings
from typing import List, Dict, Any

# 요청 시 SSL 인증서 검증 경고 비활성화
warnings.filterwarnings('ignore', category=requests.packages.urllib3.exceptions.InsecureRequestWarning)

# ====================================================
# 🌟 설정 변수
# ====================================================

# 🚩 1. 입력 CSV 파일명
LEASE_INPUT_CSV = "lh_lease_notices_F.csv"
SALE_INPUT_CSV = "lh_sale_notices_F.csv"

# 🚩 2. 출력 다운로드 폴더명 (요청하신 lh_downloads_F로 변경)
DOWNLOAD_DIR = "lh_downloads_F"

# 🚩 3. 출력 매핑 CSV 파일명 (요청하신 lh_...-download_F.csv로 변경)
LEASE_OUTPUT_MAP_CSV = "lh_lease_notices-download_F.csv"
SALE_OUTPUT_MAP_CSV = "lh_sale_notices-download_F.csv"

# 🚩 4. 허용 확장자
ALLOWED_EXTENSIONS = ('.pdf', '.xls', '.xlsx')

# 🚩 5. LH 서버 인증 핵심 쿠키 (참고 코드 사용)
FIXED_COOKIE_STRING = (
    "WMONID=Dn_IJtMsDIn; _gid=GA1.3.79854735.1763460967; _ga_6PG7CE7G5G=GS2.1.s1763460966$o8$g1$t1763460986$j40$l0$h1591851588; "
    "_ga=GA1.3.411649994.1763000357; JSESSIONID=x1eWdz56x2HLpWxU9Joduu6f_OfMRWq1NiMLTZXVm6WZlm9iRMUg!271591710!1038351822; "
    "quickMenuStts=Y; quickMenuSttsAuto=Y; NetFunnel_ID="
)

# LH 파일 다운로드 URL 패턴 및 정규식
LH_DOWNLOAD_PATH = "/lhapply/lhFile.do"
JS_PATTERN = re.compile(r"javascript:fileDownLoad\(['\"]?(\d+)['\"]?\);", re.IGNORECASE)
EXT_PATTERN = re.compile(r'\.([a-zA-Z0-9]+)$')
ID_NUMERIC_PATTERN = re.compile(r'LH_(lease|sale)_(\d+)$') # ID에서 숫자 부분 추출용

# ====================================================
# 1단계: 다운로드 관련 함수
# ====================================================

def parse_cookie_string(cookie_string: str) -> Dict[str, str]:
    """쿠키 문자열을 딕셔너리로 변환"""
    cookies = {}
    for item in cookie_string.split(';'):
        if '=' in item:
            key, value = item.strip().split('=', 1)
            cookies[key] = value
    return cookies

def get_session_headers(referer_url: str, cookie_string: str) -> tuple[Dict[str, str], Dict[str, str]]:
    """요청에 필요한 헤더와 쿠키 딕셔너리를 생성"""
    fixed_user_agent = "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Mobile Safari/537.36"
    headers = {
        'User-Agent': fixed_user_agent,
        'Referer': referer_url,
        'Host': 'apply.lh.or.kr',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    }
    cookies = parse_cookie_string(cookie_string)
    return headers, cookies

def sanitize_filename(name: str) -> str:
    """파일명 특수문자 제거 및 '바로가기' 텍스트 정리"""
    name = re.sub(r'[\\/:*?"<>|]', '_', name)
    name = re.sub(r'바\s*로\s*보\s*기\s*\d*|\s*바로가기\s*\d*', '', name, flags=re.IGNORECASE).strip()
    return name[:150].strip()

def download_file(download_url: str, file_id: str, pan_id: str, referer_url: str, raw_filename: str) -> tuple[bool, str, int]:
    """
    첨부파일 하나를 다운로드합니다.
    반환값: (성공 여부, 저장된 파일명, 파일 크기)
    """
    headers, cookies = get_session_headers(referer_url, FIXED_COOKIE_STRING)
    download_params = {'fileid': file_id}

    try:
        file_response = requests.get(
            download_url,
            params=download_params,
            headers=headers,
            cookies=cookies,
            stream=True,
            timeout=30,
            verify=False
        )
        file_response.raise_for_status()

        file_content = file_response.content
        actual_size = len(file_content)

        if actual_size < 1024 and actual_size > 0:
            print(f"  ❌ 경고: 파일 내용이 너무 작습니다. (크기: {actual_size} bytes)")
            return False, "", 0
        elif actual_size == 0:
            print(f"  ❌ 경고: 파일 다운로드 실패. 빈 응답 감지.")
            return False, "", 0
            
        # 1. 확장자 추출
        file_ext_match = EXT_PATTERN.search(raw_filename.lower())
        file_ext = file_ext_match.group(0) if file_ext_match else ".dat"

        # 2. 파일명 정리 및 최종 파일명 생성
        clean_name_base = raw_filename.replace(file_ext, '').strip()
        clean_name = sanitize_filename(clean_name_base)
        
        # (ID)로 시작하는 형식으로 저장: 예) (LH_lease_1)_공고명.pdf
        target_filename = f"({pan_id})_{clean_name}{file_ext}"

        # 3. 파일명 충돌 처리
        save_path = os.path.join(DOWNLOAD_DIR, target_filename)
        collision = 1
        while os.path.exists(save_path):
            base, ext = os.path.splitext(target_filename)
            base_no_collision = re.sub(r'(\s*\(\d+\))?$', '', base)
            target_filename = f"{base_no_collision}({collision}){ext}"
            save_path = os.path.join(DOWNLOAD_DIR, target_filename)
            collision += 1

        # 4. 파일 저장
        with open(save_path, 'wb') as f:
            f.write(file_content)

        return True, target_filename, actual_size

    except requests.exceptions.RequestException as err:
        print(f"  ❌ 요청 오류: {err}")
        return False, "", 0
    except Exception as e:
        print(f"  ❌ 일반 오류: {e}")
        return False, "", 0

def process_notice_for_download(item: Dict[str, str], total_records: int, index: int) -> List[Dict[str, str]]:
    """단일 공고 URL에서 첨부파일을 추출하고 다운로드합니다. 다운로드된 파일 정보를 반환합니다."""
    pan_id = item['ID']
    base_url = item['URL']
    pan_name = item.get('공고명', '공고명_정보없음') # 공고명 컬럼이 없는 경우 대비

    parsed_uri = urlparse(base_url)
    domain_base = f"{parsed_uri.scheme}://{parsed_uri.netloc}"
    download_url = f"{domain_base}{LH_DOWNLOAD_PATH}"
    
    downloaded_files_info = []

    print(f"\n--- {index}/{total_records}번째 공고 ({pan_id}: {pan_name[:20]}...) 처리 중 ---")

    headers, cookies = get_session_headers(base_url, FIXED_COOKIE_STRING)
    try:
        response = requests.get(base_url, headers=headers, cookies=cookies, timeout=10, verify=False)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"  ❌ 공고 접속 실패: {e}")
        return downloaded_files_info

    soup = BeautifulSoup(response.content, 'html.parser')
    attachment_data = []

    for link in soup.find_all('a', href=True):
        href = link.get('href', '')
        match_js = JS_PATTERN.search(href)

        if match_js:
            file_id = match_js.group(1)
            raw_filename = link.text.strip()

            file_ext_match = EXT_PATTERN.search(raw_filename.lower())
            file_ext = file_ext_match.group(0) if file_ext_match else None

            if file_ext and file_ext in ALLOWED_EXTENSIONS:
                attachment_data.append({
                    'id': file_id,
                    'name': raw_filename
                })

    if not attachment_data:
        print("  ℹ️ 다운로드할 유효한 첨부파일을 찾지 못했습니다.")
        return downloaded_files_info

    files_on_this_page = 0
    for data in attachment_data:
        success, target_filename, size = download_file(
            download_url,
            data['id'],
            pan_id,
            base_url,
            data['name']
        )
        if success:
            print(f"  ✅ 성공: {target_filename} (크기: {size:,} bytes)")
            downloaded_files_info.append({
                'ID': pan_id,
                '파일명': target_filename
            })
            files_on_this_page += 1
        time.sleep(0.1) # 서버 부하 방지

    print(f"  ✨ {files_on_this_page}개의 첨부파일 다운로드 시도 완료.")
    return downloaded_files_info


# ====================================================
# 2단계: 매핑 CSV 생성 관련 함수
# ====================================================

def create_mapping_csv(directory: str, prefix_type: str, output_file: str) -> int:
    """
    다운로드 폴더 내 파일명을 분석하여 ID와 파일명 매핑 CSV를 생성하고,
    ID의 숫자 순서대로 오름차순 정렬하여 저장합니다.
    """
    prefix_to_map = f"LH_{prefix_type}_"
    print(f"\n--- 🛠️ {prefix_type.upper()} 파일 ID 매핑 CSV 생성 시작 (파일 분석) ---")

    if not os.path.exists(directory):
        print(f"🚨 오류: 다운로드 폴더를 찾을 수 없습니다: {directory}")
        return 0

    # ID 전체 문자열을 추출하기 위한 정규 표현식 패턴 (예: (LH_lease_123))
    id_full_pattern = re.compile(rf'\({re.escape(prefix_to_map)}\d+\)')
    
    mapping_data: List[Dict[str, Any]] = []
    mapped_count = 0
    
    for filename in os.listdir(directory):
        # 파일명이 (LH_lease_ 또는 (LH_sale_ 로 시작하는지 확인
        if filename.startswith(f"({prefix_to_map}"):
            try:
                # 1. 파일명에서 ID 문자열을 찾습니다.
                match = id_full_pattern.match(filename)
                
                if match:
                    full_id_with_paren = match.group(0)
                    extracted_id = full_id_with_paren.strip('()')
                    
                    # 2. 정렬을 위해 ID에서 숫자 부분만 추출 (예: LH_lease_123 -> 123)
                    numeric_match = ID_NUMERIC_PATTERN.match(extracted_id)
                    numeric_id = int(numeric_match.group(2)) if numeric_match else 0
                    
                    # 3. 데이터 리스트에 추가 (정렬을 위한 임시 컬럼 포함)
                    mapping_data.append({
                        'ID': extracted_id,
                        '파일명': filename,
                        'SortIndex': numeric_id
                    })
                    mapped_count += 1
            except Exception as e:
                print(f"  ❌ 파일 분석 실패 ({filename}): {e}")

    if not mapping_data:
        print(f"⚠️ {prefix_to_map} 패턴에 일치하는 파일을 찾지 못했습니다. CSV 파일을 생성하지 않습니다.")
        return 0

    # 4. DataFrame 생성, 정렬 및 CSV 저장
    df = pd.DataFrame(mapping_data)
    
    # SortIndex(숫자)를 기준으로 오름차순 정렬
    df = df.sort_values(by='SortIndex', ascending=True)
    
    # 임시로 사용한 SortIndex 컬럼 제거
    df = df.drop(columns=['SortIndex'])
    
    try:
        df.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"✅ ID 매핑 CSV 파일 생성 및 정렬 완료: {output_file}")
        print(f"   총 {mapped_count}개의 행이 기록되었습니다.")
        return mapped_count
    except Exception as e:
        print(f"🚨 CSV 파일 저장 중 오류 발생: {e}")
        return 0


# ====================================================
# 🌟 통합 실행 함수: download()
# ====================================================

def download():
    """
    LH 공고 CSV 파일을 읽어 첨부파일을 다운로드하고,
    다운로드된 파일의 ID와 파일명을 매핑한 CSV를 생성하는 통합 함수.
    """
    start_time = time.time()
    all_download_info: List[Dict[str, str]] = []
    total_downloaded_count = 0

    # 1. 다운로드 폴더 생성
    if not os.path.exists(DOWNLOAD_DIR):
        os.makedirs(DOWNLOAD_DIR)
        print(f"✅ 다운로드 폴더 생성 완료: {DOWNLOAD_DIR}")
    print("\n" + "="*50)
    print("🚀 1단계: LH 공고 첨부파일 다운로드 시작")
    print("="*50)

    # 처리할 파일 목록 정의
    input_files = [
        (LEASE_INPUT_CSV, "임대 (LEASE)"),
        (SALE_INPUT_CSV, "분양 (SALE)")
    ]

    for input_file, file_type_name in input_files:
        print(f"\n--- {file_type_name} 파일 ({input_file}) 처리 ---")
        try:
            df = pd.read_csv(input_file)
            # 다운로드에 필요한 컬럼: 'ID', 'URL', '공고명'
            required_cols = ['ID', 'URL']
            if '공고명' in df.columns:
                required_cols.append('공고명')
            
            all_url_data = df[required_cols].to_dict('records')

        except FileNotFoundError:
            print(f"🚨 오류: 입력 CSV 파일을 찾을 수 없습니다: {input_file}. 건너뜁니다.")
            continue
        except Exception as e:
            print(f"🚨 CSV 파일 로드 중 오류 발생: {e}. 건너뜁니다.")
            continue

        total_records = len(all_url_data)
        print(f"총 {total_records}개의 공고 URL에 대해 파일 다운로드를 시도합니다.")

        current_download_count = 0
        for i, item in enumerate(all_url_data):
            # item 딕셔너리에 '공고명'이 없을 경우를 대비해 get으로 처리
            downloaded_files = process_notice_for_download(item, total_records, i + 1)
            
            current_download_count += len(downloaded_files)
            time.sleep(0.5) # 공고별 딜레이
        
        total_downloaded_count += current_download_count
        print(f"🎉 {file_type_name} 파일 다운로드 시도 완료. 성공: {current_download_count}개.")


    print("\n" + "="*50)
    print("✅ 2단계: ID-파일명 매핑 CSV 생성 시작")
    print("="*50)
    
    # 2. 임대(lease) 매핑 CSV 생성
    lease_mapped_count = create_mapping_csv(
        DOWNLOAD_DIR, 
        "lease", 
        LEASE_OUTPUT_MAP_CSV
    )
    
    # 3. 분양(sale) 매핑 CSV 생성
    sale_mapped_count = create_mapping_csv(
        DOWNLOAD_DIR, 
        "sale", 
        SALE_OUTPUT_MAP_CSV
    )

    end_time = time.time()
    print("\n\n" + "="*50)
    print("🚀 통합 다운로드 및 매핑 작업 최종 완료")
    print(f"총 다운로드 성공 파일 수: {total_downloaded_count}개")
    print(f"다운로드 폴더: {DOWNLOAD_DIR}")
    print(f"매핑 파일 생성 (LEASE): {LEASE_OUTPUT_MAP_CSV} ({lease_mapped_count}개 행)")
    print(f"매핑 파일 생성 (SALE): {SALE_OUTPUT_MAP_CSV} ({sale_mapped_count}개 행)")
    print(f"총 소요 시간: {end_time - start_time:.2f}초")
    print("="*50)


# ==================================
# 5. 실행
# ==================================

if __name__ == "__main__":
    download()

✅ 다운로드 폴더 생성 완료: lh_downloads_F

🚀 1단계: LH 공고 첨부파일 다운로드 시작

--- 임대 (LEASE) 파일 (lh_lease_notices_F.csv) 처리 ---
총 1645개의 공고 URL에 대해 파일 다운로드를 시도합니다.

--- 1/1645번째 공고 (LH_lease_1: [선계약후검증, 동호지정] 도청이전신...) 처리 중 ---
  ✅ 성공: (LH_lease_1)_도청이전신도시RH-11BL국민임대입주자모집공고('24.11.01).pdf (크기: 496,814 bytes)
  ✅ 성공: (LH_lease_1)_도청이전신도시RH-11BL국민_팜플렛.pdf (크기: 4,609,378 bytes)
  ✅ 성공: (LH_lease_1)_[■필독]공사양식작성방법.pdf (크기: 493,471 bytes)
  ✅ 성공: (LH_lease_1)_[■필독]제출서류안내.pdf (크기: 122,587 bytes)
  ✨ 4개의 첨부파일 다운로드 시도 완료.

--- 2/1645번째 공고 (LH_lease_2: 태백철암1 국민임대주택 예비입주자 모...) 처리 중 ---
  ✅ 성공: (LH_lease_2)_2024.11.01태백철암국민임대주택모집공고문.pdf (크기: 1,090,409 bytes)
  ✨ 1개의 첨부파일 다운로드 시도 완료.

--- 3/1645번째 공고 (LH_lease_3: [선계약후검증, 동호지정] 도청이전신...) 처리 중 ---
  ✅ 성공: (LH_lease_3)_도청이전신도시RH10-2BL국민임대입주자모집공고('24.11.01).pdf (크기: 503,683 bytes)
  ✅ 성공: (LH_lease_3)_도청이전신도시RH10-2국민_팜플렛.pdf (크기: 21,011,977 bytes)
  ✅ 성공: (LH_lease_3)_[■필독]공사양식작성방법.pdf (크기: 493,471 bytes)
  ✅ 성공: (LH_lease_3)_[■필독]제출서류안내.pdf (크기: 122,587 bytes)
  ✨ 4개

# 업데이트 함수